# Data Understanding
This notebook was built for validating my understanding of the available data.

In [1]:
import os
from working_dir import set_wd
set_wd()
os.getcwd()

'/Users/tales.pimentel/ds/kaggle/football-match-prediction'

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataUnderstanding").getOrCreate()

In [3]:
import pyspark.sql.functions as f

from src.dao import dao_raw
from src.utils import dflib, stats, pretties

In [4]:
pretties.max_data_frame_columns()

#### Predict the probabilities for each match outcome in the target columns: the probabilities of home to win, draw, and away to win.

In [5]:
train = dao_raw.load_parse_train_data(spark)
test = dao_raw.load_parse_test_data(spark)
scores = dao_raw.load_parse_scores_data(spark)

print(f"train: {dflib.shape(train)}")
print(f"test: {dflib.shape(test)}")
print(f"scores: {dflib.shape(scores)}")

train: (110938, 190)
test: (72711, 189)
scores: (110938, 3)


In [6]:
train = scores.select(["id", "score"]).join(train, on="id", how="inner")
print(f"train: {dflib.shape(train)}")

train: (110938, 191)


In [7]:
stats.freq(train, "league_name", 6).show()

+--------------------+--------+--------+
|         league_name|Absolute|Relative|
+--------------------+--------+--------+
|              1. SNL|     237|0.002136|
|              2. SNL|     117|0.001055|
|        Youth League|     467| 0.00421|
|Japan Football Le...|      78| 7.03E-4|
|  Reserve Pro League|      56| 5.05E-4|
|             Serie A|     985|0.008879|
|            Super D1|     170|0.001532|
|Vanarama National...|     171|0.001541|
|              Liga 2|     303|0.002731|
|        Slovakia Cup|      63| 5.68E-4|
|          Uae League|     245|0.002208|
|Gamma Ethniki Gro...|      13| 1.17E-4|
|   Serie D: Girone C|     365| 0.00329|
|  1st National Women|       7|  6.3E-5|
|    Oberliga: Bremen|     102| 9.19E-4|
|           Elite One|     239|0.002154|
|            National|     369|0.003326|
|Gamma Ethniki Gro...|      21| 1.89E-4|
|  Spain Youth League|     605|0.005453|
|Vanarama National...|     343|0.003092|
+--------------------+--------+--------+
only showing top

In [8]:
stats.freq(train, "home_team_history_is_play_home_1", 6).show()

+--------------------------------+--------+--------+
|home_team_history_is_play_home_1|Absolute|Relative|
+--------------------------------+--------+--------+
|                            null|  110938|     1.0|
+--------------------------------+--------+--------+



In [9]:
train.limit(5).toPandas()

,id,score,target,home_team_name,away_team_name,match_date,league_name,league_id,is_cup,home_team_coach_id,away_team_coach_id,home_team_history_match_date_1,home_team_history_match_date_2,home_team_history_match_date_3,home_team_history_match_date_4,home_team_history_match_date_5,home_team_history_match_date_6,home_team_history_match_date_7,home_team_history_match_date_8,home_team_history_match_date_9,home_team_history_match_date_10,home_team_history_is_play_home_1,home_team_history_is_play_home_2,home_team_history_is_play_home_3,home_team_history_is_play_home_4,home_team_history_is_play_home_5,home_team_history_is_play_home_6,home_team_history_is_play_home_7,home_team_history_is_play_home_8,home_team_history_is_play_home_9,home_team_history_is_play_home_10,home_team_history_is_cup_1,home_team_history_is_cup_2,home_team_history_is_cup_3,home_team_history_is_cup_4,home_team_history_is_cup_5,home_team_history_is_cup_6,home_team_history_is_cup_7,home_team_history_is_cup_8,home_team_history_is_cup_9,home_team_history_is_cup_10,home_team_history_goal_1,home_team_history_goal_2,home_team_history_goal_3,home_team_history_goal_4,home_team_history_goal_5,home_team_history_goal_6,home_team_history_goal_7,home_team_history_goal_8,home_team_history_goal_9,home_team_history_goal_10,home_team_history_opponent_goal_1,home_team_history_opponent_goal_2,home_team_history_opponent_goal_3,home_team_history_opponent_goal_4,home_team_history_opponent_goal_5,home_team_history_opponent_goal_6,home_team_history_opponent_goal_7,home_team_history_opponent_goal_8,home_team_history_opponent_goal_9,home_team_history_opponent_goal_10,home_team_history_rating_1,home_team_history_rating_2,home_team_history_rating_3,home_team_history_rating_4,home_team_history_rating_5,home_team_history_rating_6,home_team_history_rating_7,home_team_history_rating_8,home_team_history_rating_9,home_team_history_rating_10,home_team_history_opponent_rating_1,home_team_history_opponent_rating_2,home_team_history_opponent_rating_3,home_team_history_opponent_rating_4,home_team_history_opponent_rating_5,home_team_history_opponent_rating_6,home_team_history_opponent_rating_7,home_team_history_opponent_rating_8,home_team_history_opponent_rating_9,home_team_history_opponent_rating_10,home_team_history_coach_1,home_team_history_coach_2,home_team_history_coach_3,home_team_history_coach_4,home_team_history_coach_5,home_team_history_coach_6,home_team_history_coach_7,home_team_history_coach_8,home_team_history_coach_9,home_team_history_coach_10,home_team_history_league_id_1,home_team_history_league_id_2,home_team_history_league_id_3,home_team_history_league_id_4,home_team_history_league_id_5,home_team_history_league_id_6,home_team_history_league_id_7,home_team_history_league_id_8,home_team_history_league_id_9,home_team_history_league_id_10,away_team_history_match_date_1,away_team_history_match_date_2,away_team_history_match_date_3,away_team_history_match_date_4,away_team_history_match_date_5,away_team_history_match_date_6,away_team_history_match_date_7,away_team_history_match_date_8,away_team_history_match_date_9,away_team_history_match_date_10,away_team_history_is_play_home_1,away_team_history_is_play_home_2,away_team_history_is_play_home_3,away_team_history_is_play_home_4,away_team_history_is_play_home_5,away_team_history_is_play_home_6,away_team_history_is_play_home_7,away_team_history_is_play_home_8,away_team_history_is_play_home_9,away_team_history_is_play_home_10,away_team_history_is_cup_1,away_team_history_is_cup_2,away_team_history_is_cup_3,away_team_history_is_cup_4,away_team_history_is_cup_5,away_team_history_is_cup_6,away_team_history_is_cup_7,away_team_history_is_cup_8,away_team_history_is_cup_9,away_team_history_is_cup_10,away_team_history_goal_1,away_team_history_goal_2,away_team_history_goal_3,away_team_history_goal_4,away_team_history_goal_5,away_team_history_goal_6,away_team_history_goal_7,away_team_history_goal_8,away_team_history_goal_9,away_team_history_goal_10,away_team_hi

# Case Study: Sevilla

### Train

In [10]:
sevilla = train.filter(((train["home_team_name"] == "Sevilla") |
                        (train["away_team_name"] == "Sevilla"))).sort("match_date", ascending=False)

sevilla_laliga = sevilla.filter(sevilla["league_name"] == "La Liga").sort("match_date", ascending=False)

In [11]:
display(stats.freq(sevilla, "league_name").toPandas())
display(stats.freq(sevilla, "is_cup").toPandas())

,league_name,Absolute,Relative
0,Copa Del Rey,11,0.127907
1,La Liga,57,0.662791
2,Europa League,7,0.081395
3,Club Friendlies,2,0.023256
4,Uefa Super Cup,1,0.011628
5,Champions League,8,0.093023


,is_cup,Absolute,Relative
0,False,59,0.686047
1,True,27,0.313953


In [12]:
sevilla_laliga.limit(11).toPandas()

,id,score,target,home_team_name,away_team_name,match_date,league_name,league_id,is_cup,home_team_coach_id,away_team_coach_id,home_team_history_match_date_1,home_team_history_match_date_2,home_team_history_match_date_3,home_team_history_match_date_4,home_team_history_match_date_5,home_team_history_match_date_6,home_team_history_match_date_7,home_team_history_match_date_8,home_team_history_match_date_9,home_team_history_match_date_10,home_team_history_is_play_home_1,home_team_history_is_play_home_2,home_team_history_is_play_home_3,home_team_history_is_play_home_4,home_team_history_is_play_home_5,home_team_history_is_play_home_6,home_team_history_is_play_home_7,home_team_history_is_play_home_8,home_team_history_is_play_home_9,home_team_history_is_play_home_10,home_team_history_is_cup_1,home_team_history_is_cup_2,home_team_history_is_cup_3,home_team_history_is_cup_4,home_team_history_is_cup_5,home_team_history_is_cup_6,home_team_history_is_cup_7,home_team_history_is_cup_8,home_team_history_is_cup_9,home_team_history_is_cup_10,home_team_history_goal_1,home_team_history_goal_2,home_team_history_goal_3,home_team_history_goal_4,home_team_history_goal_5,home_team_history_goal_6,home_team_history_goal_7,home_team_history_goal_8,home_team_history_goal_9,home_team_history_goal_10,home_team_history_opponent_goal_1,home_team_history_opponent_goal_2,home_team_history_opponent_goal_3,home_team_history_opponent_goal_4,home_team_history_opponent_goal_5,home_team_history_opponent_goal_6,home_team_history_opponent_goal_7,home_team_history_opponent_goal_8,home_team_history_opponent_goal_9,home_team_history_opponent_goal_10,home_team_history_rating_1,home_team_history_rating_2,home_team_history_rating_3,home_team_history_rating_4,home_team_history_rating_5,home_team_history_rating_6,home_team_history_rating_7,home_team_history_rating_8,home_team_history_rating_9,home_team_history_rating_10,home_team_history_opponent_rating_1,home_team_history_opponent_rating_2,home_team_history_opponent_rating_3,home_team_history_opponent_rating_4,home_team_history_opponent_rating_5,home_team_history_opponent_rating_6,home_team_history_opponent_rating_7,home_team_history_opponent_rating_8,home_team_history_opponent_rating_9,home_team_history_opponent_rating_10,home_team_history_coach_1,home_team_history_coach_2,home_team_history_coach_3,home_team_history_coach_4,home_team_history_coach_5,home_team_history_coach_6,home_team_history_coach_7,home_team_history_coach_8,home_team_history_coach_9,home_team_history_coach_10,home_team_history_league_id_1,home_team_history_league_id_2,home_team_history_league_id_3,home_team_history_league_id_4,home_team_history_league_id_5,home_team_history_league_id_6,home_team_history_league_id_7,home_team_history_league_id_8,home_team_history_league_id_9,home_team_history_league_id_10,away_team_history_match_date_1,away_team_history_match_date_2,away_team_history_match_date_3,away_team_history_match_date_4,away_team_history_match_date_5,away_team_history_match_date_6,away_team_history_match_date_7,away_team_history_match_date_8,away_team_history_match_date_9,away_team_history_match_date_10,away_team_history_is_play_home_1,away_team_history_is_play_home_2,away_team_history_is_play_home_3,away_team_history_is_play_home_4,away_team_history_is_play_home_5,away_team_history_is_play_home_6,away_team_history_is_play_home_7,away_team_history_is_play_home_8,away_team_history_is_play_home_9,away_team_history_is_play_home_10,away_team_history_is_cup_1,away_team_history_is_cup_2,away_team_history_is_cup_3,away_team_history_is_cup_4,away_team_history_is_cup_5,away_team_history_is_cup_6,away_team_history_is_cup_7,away_team_history_is_cup_8,away_team_history_is_cup_9,away_team_history_is_cup_10,away_team_history_goal_1,away_team_history_goal_2,away_team_history_goal_3,away_team_history_goal_4,away_team_history_goal_5,away_team_history_goal_6,away_team_history_goal_7,away_team_history_goal_8,away_team_history_goal_9,away_team_history_goal_10,away_team_hi

In [13]:
last_n_match = 1
sevilla_laliga.select(["id", "target", "score", "match_date", "home_team_name", "away_team_name", 
                      f"away_team_history_match_date_{last_n_match}", 
                      f"away_team_history_is_play_home_{last_n_match}",
                      f"away_team_history_goal_{last_n_match}",
                      f"away_team_history_opponent_goal_{last_n_match}",
                      f"away_team_history_rating_{last_n_match}",
                      f"away_team_history_rating_{last_n_match + 1}",
                      f"away_team_history_rating_{last_n_match + 2}",
                      f"away_team_history_rating_{last_n_match + 3}",
                      f"home_team_history_rating_{last_n_match}",
                      f"home_team_history_rating_{last_n_match + 1}",
                      f"home_team_history_rating_{last_n_match + 2}",
                      f"home_team_history_rating_{last_n_match + 3}"]).limit(8).toPandas()

,id,target,score,match_date,home_team_name,away_team_name,away_team_history_match_date_1,away_team_history_is_play_home_1,away_team_history_goal_1,away_team_history_opponent_goal_1,away_team_history_rating_1,away_team_history_rating_2,away_team_history_rating_3,away_team_history_rating_4,home_team_history_rating_1,home_team_history_rating_2,home_team_history_rating_3,home_team_history_rating_4
0,17055666,home,2-1,2021-04-25 16:30:00,Sevilla,Granada,2021-04-22 19:00:00,None,4.0,1.0,5.039163888888889,3.5954750000000004,4.972172222222222,3.5298233333333324,6.783431818181818,5.473114285714286,6.478452777777778,4.888375
1,17055657,away,0-1,2021-04-21 17:00:00,Levante,Sevilla,2021-04-18 12:00:00,None,2.0,1.0,5.473114285714286,6.478452777777778,4.888375,6.492458333333332,4.7946307692307695,5.0189055555555555,6.555246666666666,4.448794444444444
2,17055680,away,1-2,2021-04-18 12:00:00,Real Sociedad,Sevilla,2021-04-12 19:00:00,None,4.0,3.0,6.478452777777778,4.888375,6.492458333333332,10.316407142857143,6.892741666666665,5.80088,5.23726875,5.475752083333333
3,17055643,away,3-4,2021-04-12 19:00:00,Celta de Vigo,Sevilla,2021-04-04 19:00:00,None,1.0,0.0,4.888375,6.492458333333332,10.316407142857143,6.0071111111111115,5.590813333333332,4.562364705882353,5.410818749999999,6.089327777777779
4,17055639,home,1-0,2021-04-04 19:00:00,Sevilla,Atlético Madrid,2021-03-21 17:30:00,None,1.0,0.0,8.630291666666668,4.651570000000001,6.034585294117647,7.3606375,6.492458333333332,10.316407142857143,6.0071111111111115,5.894426666666668
5,17055629,draw,1-1,2021-03-20 20:00:00,Real Valladolid,Sevilla,2021-03-17 18:00:00,None,2.0,0.0,10.316407142857143,6.0071111111111115,5.894426666666668,8.676955555555555,4.136038235294118,4.453936111111111,4.478336111111111,3.6886590909090913
6,17055369,home,2-0,2021-03-17 18:00:00,Sevilla,Elche,2021-03-13 15:15:00,None,1.0,2.0,2.9914941176470586,2.936166666666667,3.853858823529412,3.4267428571428575,6.0071111111111115,5.894426666666668,8.676955555555555,4.5483777777777785
7,17055620,home,1-0,2021-03-14 20:00:00,Sevilla,Real Betis,2021-03-08 20:00:00,None,3.0,2.0,6.6376124999999995,6.579238235294117,5.27759,4.046854545454545,5.894426666666668,8.676955555555555,4.5483777777777785,5.240755882352941


Ratings

In [14]:
display(sevilla_laliga.filter(sevilla_laliga["id"] == "17055643").select(["away_team_history_rating_3"]).toPandas())
display(sevilla_laliga.filter(sevilla_laliga["id"] == "17055639").select(["home_team_history_rating_2"]).toPandas())

,away_team_history_rating_3
0,10.316407142857143


,home_team_history_rating_2
0,10.316407142857143


Goals

In [15]:
sevilla_laliga.filter(sevilla_laliga["id"] == "17055666").select(["home_team_history_opponent_goal_2", "home_team_history_goal_2"]).toPandas()

,home_team_history_opponent_goal_2,home_team_history_goal_2
0,1.0,2.0


### Test

In [16]:
laliga_test = test.filter(test["league_name"] == "La Liga").sort("match_date", ascending=False)

In [17]:
laliga_test.limit(5).toPandas()

,id,home_team_name,away_team_name,match_date,league_name,league_id,is_cup,home_team_coach_id,away_team_coach_id,home_team_history_match_date_1,home_team_history_match_date_2,home_team_history_match_date_3,home_team_history_match_date_4,home_team_history_match_date_5,home_team_history_match_date_6,home_team_history_match_date_7,home_team_history_match_date_8,home_team_history_match_date_9,home_team_history_match_date_10,home_team_history_is_play_home_1,home_team_history_is_play_home_2,home_team_history_is_play_home_3,home_team_history_is_play_home_4,home_team_history_is_play_home_5,home_team_history_is_play_home_6,home_team_history_is_play_home_7,home_team_history_is_play_home_8,home_team_history_is_play_home_9,home_team_history_is_play_home_10,home_team_history_is_cup_1,home_team_history_is_cup_2,home_team_history_is_cup_3,home_team_history_is_cup_4,home_team_history_is_cup_5,home_team_history_is_cup_6,home_team_history_is_cup_7,home_team_history_is_cup_8,home_team_history_is_cup_9,home_team_history_is_cup_10,home_team_history_goal_1,home_team_history_goal_2,home_team_history_goal_3,home_team_history_goal_4,home_team_history_goal_5,home_team_history_goal_6,home_team_history_goal_7,home_team_history_goal_8,home_team_history_goal_9,home_team_history_goal_10,home_team_history_opponent_goal_1,home_team_history_opponent_goal_2,home_team_history_opponent_goal_3,home_team_history_opponent_goal_4,home_team_history_opponent_goal_5,home_team_history_opponent_goal_6,home_team_history_opponent_goal_7,home_team_history_opponent_goal_8,home_team_history_opponent_goal_9,home_team_history_opponent_goal_10,home_team_history_rating_1,home_team_history_rating_2,home_team_history_rating_3,home_team_history_rating_4,home_team_history_rating_5,home_team_history_rating_6,home_team_history_rating_7,home_team_history_rating_8,home_team_history_rating_9,home_team_history_rating_10,home_team_history_opponent_rating_1,home_team_history_opponent_rating_2,home_team_history_opponent_rating_3,home_team_history_opponent_rating_4,home_team_history_opponent_rating_5,home_team_history_opponent_rating_6,home_team_history_opponent_rating_7,home_team_history_opponent_rating_8,home_team_history_opponent_rating_9,home_team_history_opponent_rating_10,home_team_history_coach_1,home_team_history_coach_2,home_team_history_coach_3,home_team_history_coach_4,home_team_history_coach_5,home_team_history_coach_6,home_team_history_coach_7,home_team_history_coach_8,home_team_history_coach_9,home_team_history_coach_10,home_team_history_league_id_1,home_team_history_league_id_2,home_team_history_league_id_3,home_team_history_league_id_4,home_team_history_league_id_5,home_team_history_league_id_6,home_team_history_league_id_7,home_team_history_league_id_8,home_team_history_league_id_9,home_team_history_league_id_10,away_team_history_match_date_1,away_team_history_match_date_2,away_team_history_match_date_3,away_team_history_match_date_4,away_team_history_match_date_5,away_team_history_match_date_6,away_team_history_match_date_7,away_team_history_match_date_8,away_team_history_match_date_9,away_team_history_match_date_10,away_team_history_is_play_home_1,away_team_history_is_play_home_2,away_team_history_is_play_home_3,away_team_history_is_play_home_4,away_team_history_is_play_home_5,away_team_history_is_play_home_6,away_team_history_is_play_home_7,away_team_history_is_play_home_8,away_team_history_is_play_home_9,away_team_history_is_play_home_10,away_team_history_is_cup_1,away_team_history_is_cup_2,away_team_history_is_cup_3,away_team_history_is_cup_4,away_team_history_is_cup_5,away_team_history_is_cup_6,away_team_history_is_cup_7,away_team_history_is_cup_8,away_team_history_is_cup_9,away_team_history_is_cup_10,away_team_history_goal_1,away_team_history_goal_2,away_team_history_goal_3,away_team_history_goal_4,away_team_history_goal_5,away_team_history_goal_6,away_team_history_goal_7,away_team_history_goal_8,away_team_history_goal_9,away_team_history_goal_10,away_team_history_opponen